In [1]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as torch_data

In [2]:
from torch import cuda

if cuda.is_available():
    device = 'cuda'
    torch.cuda.manual_seed_all(seed)
    print("running on GPU:", torch.cuda.get_device_name(0))
else:
    print("running on CPU")

running on CPU


In [3]:
from CommentsDatasets import * # torch dataset setup
from models import * # all our LSTM based models

In [4]:
train_comment_array = torch.load('./StoredTensors/w2v_train_comment_array.pt')
train_title_array = torch.load('./StoredTensors/w2v_train_title_array.pt')
train_labels = torch.load('./StoredTensors/w2v_train_labels.pt')

test_comment_array = torch.load('./StoredTensors/w2v_test_comment_array.pt')
test_title_array = torch.load('./StoredTensors/w2v_test_title_array.pt')
test_labels = torch.load('./StoredTensors/w2v_test_labels.pt')

In [5]:
print(train_comment_array.dtype)
print(type(train_comment_array))
print(train_title_array.dtype)
print(type(train_title_array))

float32
<class 'numpy.ndarray'>
float32
<class 'numpy.ndarray'>


In [6]:
"""
input data should be: (batch_size, max_length, embed_dim)
labels should be: (batch_size,)
"""
print(train_comment_array.shape, train_title_array.shape, train_labels.shape)
print(test_comment_array.shape, test_title_array.shape, test_labels.shape)

(1528, 244, 300) (1528, 13, 300) (1528,)
(102, 126, 300) (102, 19, 300) (102,)


In [65]:
from tqdm import tqdm

def train(data, labels, n_epochs, batch_size, modeltype, model = None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    if model is None:
        model = modeltype(bidi=False,attention=False).to(device)
        
    opt = optim.Adam(model.parameters(), lr=0.001)

    training_data = CommentsDataset(range(len(labels)), data, labels)

    loader = torch_data.DataLoader(training_data, batch_size=batch_size, shuffle=True)

    loss_fn = nn.BCELoss()

    losses = []
    for epoch in tqdm(range(n_epochs)):
        epoch_loss = 0
        for context, label in loader:
            context = context.to(device)
            label = label.to(device).type(torch.float32)

            preds = model.forward(context)

            opt.zero_grad()
            loss = loss_fn(preds, label)
            loss.backward()
            opt.step()

            epoch_loss += loss.item()
        print('Loss:', epoch_loss)
        losses.append(epoch_loss)

    print(losses)
    return model

In [75]:
model = Full_LSTM_Model(bidi=True,attention=True)
num_epochs = 5 #running on colab, use 30-40ish
trained_model = train(train_comment_array,train_labels, num_epochs, 128, Full_LSTM_Model, model)

 20%|█████████                                    | 1/5 [00:08<00:34,  8.57s/it]

Loss: 7.802475154399872


 40%|██████████████████                           | 2/5 [00:16<00:24,  8.13s/it]

Loss: 7.234598994255066


 60%|███████████████████████████                  | 3/5 [00:25<00:16,  8.46s/it]

Loss: 7.1610758900642395


 80%|████████████████████████████████████         | 4/5 [00:34<00:08,  8.63s/it]

Loss: 7.05184930562973


100%|█████████████████████████████████████████████| 5/5 [00:42<00:00,  8.48s/it]

Loss: 6.764976263046265
[7.802475154399872, 7.234598994255066, 7.1610758900642395, 7.05184930562973, 6.764976263046265]
